## Tensorflow的Layer基础知识介绍

Layer是构成Model的基本单元，简单理解成一个数据处理函数：
* Layer可以输入一个数据tensor，经过代码处理，输出一个数据Tensor
* Layer中有参数Weights，Model训练的目标，就是提供数据(features、label），优化Model中的Layer的Weights

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

### 1. Layer就像函数一样调用

In [2]:
# Dense即全连接层
layer = layers.Dense(3, activation='relu')

In [3]:
# 输入数据为一个tensor
inputs = tf.random.uniform(shape=(5, 6))
inputs

<tf.Tensor: shape=(5, 6), dtype=float32, numpy=
array([[0.33835745, 0.4028859 , 0.23511803, 0.21115911, 0.62988913,
        0.15494311],
       [0.18937635, 0.2366414 , 0.5940547 , 0.07008147, 0.7589654 ,
        0.89733934],
       [0.9675964 , 0.85580623, 0.91624475, 0.75455165, 0.7629715 ,
        0.1385355 ],
       [0.15827405, 0.9905658 , 0.9414414 , 0.7063625 , 0.31134558,
        0.9627578 ],
       [0.1599555 , 0.0049727 , 0.21972692, 0.76499987, 0.15703976,
        0.22014487]], dtype=float32)>

In [4]:
# 执行函数变换，注意输入输出的形状也变化了
layer(inputs)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0.7080624 , 0.        , 0.        ],
       [0.42272323, 0.4542595 , 0.        ],
       [1.2965715 , 0.        , 0.        ],
       [0.45373344, 0.        , 0.        ],
       [0.5377994 , 0.        , 0.        ]], dtype=float32)>

### 2. Layer本身常用的方法

In [5]:
# 虽然Layer可以像函数一样调用处理输入得到输出，但是它本身维护了可以更新的Weights参数
layer.weights

[<tf.Variable 'dense/kernel:0' shape=(6, 3) dtype=float32, numpy=
 array([[ 0.28796923, -0.13785005, -0.27724725],
        [ 0.11302459, -0.7230098 , -0.6558574 ],
        [-0.11057937, -0.6325079 , -0.57528687],
        [ 0.58731425, -0.5676677 , -0.7046772 ],
        [ 0.80964005,  0.70802915, -0.5309555 ],
        [-0.27694768,  0.59020793,  0.44910252]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [6]:
# trainable_weights会在训练期间，通过梯度下降进行更新
layer.trainable_weights

[<tf.Variable 'dense/kernel:0' shape=(6, 3) dtype=float32, numpy=
 array([[ 0.28796923, -0.13785005, -0.27724725],
        [ 0.11302459, -0.7230098 , -0.6558574 ],
        [-0.11057937, -0.6325079 , -0.57528687],
        [ 0.58731425, -0.5676677 , -0.7046772 ],
        [ 0.80964005,  0.70802915, -0.5309555 ],
        [-0.27694768,  0.59020793,  0.44910252]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [7]:
# 以含有Numpy矩阵的列表形式返回层的权重
layer.get_weights()

[array([[ 0.28796923, -0.13785005, -0.27724725],
        [ 0.11302459, -0.7230098 , -0.6558574 ],
        [-0.11057937, -0.6325079 , -0.57528687],
        [ 0.58731425, -0.5676677 , -0.7046772 ],
        [ 0.80964005,  0.70802915, -0.5309555 ],
        [-0.27694768,  0.59020793,  0.44910252]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [8]:
layer.kernel

<tf.Variable 'dense/kernel:0' shape=(6, 3) dtype=float32, numpy=
array([[ 0.28796923, -0.13785005, -0.27724725],
       [ 0.11302459, -0.7230098 , -0.6558574 ],
       [-0.11057937, -0.6325079 , -0.57528687],
       [ 0.58731425, -0.5676677 , -0.7046772 ],
       [ 0.80964005,  0.70802915, -0.5309555 ],
       [-0.27694768,  0.59020793,  0.44910252]], dtype=float32)>

In [9]:
layer.bias

<tf.Variable 'dense/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>

In [10]:
# 包含层配置的字典
layer.get_config()

{'name': 'dense',
 'trainable': True,
 'dtype': 'float32',
 'units': 3,
 'activation': 'relu',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

### 3. 自己实现Dense全连接层

Dense 实现以下操作： output = activation(dot(input, kernel) + bias)：
* kernel 是由网络层创建的权值矩阵
* bias 是其创建的偏置向量
* activation 是按逐个元素计算的激活函数

In [11]:
class SimpleDense(tf.keras.layers.Layer):
    """要实现一个自己的Layer，首先继承tf.keras.Layer"""
    
    def __init__(self, units=32, activation=tf.keras.activations.relu):
        """在这里实现跟输入数据无关的初始化，units是输出单元个数"""
        super(SimpleDense, self).__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        """这里你已经知道了输入shape，做一些其它初始化"""
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(initial_value=w_init(shape=(input_shape[-1], self.units), dtype='float32'), trainable=True)
        
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(self.units,), dtype='float32'), trainable=True)

    def call(self, inputs):
        """在这里执行真正的计算，m*n @ n*k = m*k"""
        return self.activation(tf.matmul(inputs, self.w) + self.b)

In [12]:
simpleDense = SimpleDense(3)

In [13]:
# 输入数据
inputs = tf.random.uniform(shape=(5, 6))
inputs

<tf.Tensor: shape=(5, 6), dtype=float32, numpy=
array([[0.84321594, 0.94971406, 0.00291455, 0.15217948, 0.18815064,
        0.7553489 ],
       [0.45222616, 0.8940605 , 0.66049397, 0.7730479 , 0.25303853,
        0.08674979],
       [0.04443181, 0.48009157, 0.58369625, 0.7755337 , 0.58347166,
        0.21425164],
       [0.6621847 , 0.75676584, 0.24291241, 0.1357758 , 0.78865397,
        0.6802577 ],
       [0.7083311 , 0.48510265, 0.24692857, 0.2819258 , 0.8745481 ,
        0.12379682]], dtype=float32)>

In [14]:
simpleDense(inputs)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0.04327397, 0.04678761, 0.09832035],
       [0.07879104, 0.1084666 , 0.00513381],
       [0.04440111, 0.08707336, 0.        ],
       [0.05931166, 0.06264593, 0.0342182 ],
       [0.08149881, 0.08866378, 0.        ]], dtype=float32)>